In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import cvxpy as cp
from cvxopt import matrix, solvers
import yfinance as yf
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
from scipy.stats import norm

In [2]:
# Load the stock data.
file_path = '.\data\s&p500\clean_s&p500_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)

In [3]:
stock_data.head()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,1268.800049,23.962805,37.450001,43.520000,2.669643,NaN,9.44,NaN,18.976089,6.120000,...,18.570000,58.470001,27.014999,39.288540,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,39.200001,43.830002,2.677500,NaN,9.62,NaN,19.004877,6.151111,...,18.660000,58.570000,27.264999,39.235836,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,39.709999,44.040001,2.656429,NaN,9.55,NaN,19.249577,6.061111,...,18.650000,58.279999,26.955000,38.814228,NaN,17.670740,66.407768,42.529999,77.720001,NaN
2006-01-06,1285.449951,24.785408,39.000000,43.990002,2.725000,NaN,9.75,NaN,19.619022,6.173333,...,18.719999,59.430000,27.375000,38.577076,NaN,17.620417,66.067963,44.119999,78.529999,NaN
2006-01-09,1290.150024,24.713877,38.610001,44.560001,2.716071,NaN,10.15,NaN,20.348318,6.116667,...,18.670000,59.400002,27.645000,39.104084,NaN,17.757010,68.407768,44.790001,77.879997,NaN


In [4]:
stock_data.tail(5)

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-12-23,3844.820068,149.229996,12.71,143.279999,131.860001,163.100006,381.019989,NaN,108.180000,63.380001,...,70.930000,108.680000,31.830000,14.61,109.730003,128.899994,126.690002,248.220001,48.450001,145.759995
2022-12-27,3829.250000,149.550003,12.53,145.020004,130.029999,162.990005,381.019989,NaN,108.570000,63.619999,...,71.570000,110.190002,32.070000,14.70,110.720001,129.899994,127.279999,251.000000,48.840000,145.300003
2022-12-28,3783.219971,148.089996,12.32,145.300003,126.040001,162.229996,381.019989,NaN,107.830002,62.599998,...,70.570000,108.379997,30.980000,14.37,108.940002,129.309998,125.989998,246.839996,47.970001,143.830002
2022-12-29,3849.280029,151.089996,12.70,146.309998,129.610001,162.559998,381.019989,NaN,110.309998,63.110001,...,71.070000,109.199997,32.279999,14.49,111.639999,129.990005,127.830002,257.529999,49.080002,148.149994
2022-12-30,3839.500000,149.649994,12.72,147.029999,129.929993,161.610001,381.019989,NaN,109.790001,62.779999,...,70.110001,110.300003,31.840000,14.60,110.570000,128.080002,127.500000,256.410004,49.160000,146.550003


In [5]:
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')
stock_returns.tail()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-12-23,0.005868,0.001476,0.011943,0.008446,-0.002798,-0.001041,0.0,0.0,0.001389,0.008433,...,0.012852,0.026445,0.011118,-0.003411,-0.000728,0.000621,-0.000789,0.002869,0.003521,0.005033
2022-12-27,-0.004050,0.002144,-0.014162,0.012144,-0.013878,-0.000674,0.0,0.0,0.003605,0.003787,...,0.009023,0.013894,0.007540,0.006160,0.009022,0.007758,0.004657,0.011200,0.008050,-0.003156
2022-12-28,-0.012021,-0.009763,-0.016760,0.001931,-0.030685,-0.004663,0.0,0.0,-0.006816,-0.016033,...,-0.013972,-0.016426,-0.033988,-0.022449,-0.016077,-0.004542,-0.010135,-0.016574,-0.017813,-0.010117
2022-12-29,0.017461,0.020258,0.030844,0.006951,0.028324,0.002034,0.0,0.0,0.022999,0.008147,...,0.007085,0.007566,0.041963,0.008351,0.024784,0.005259,0.014604,0.043307,0.023139,0.030035
2022-12-30,-0.002541,-0.009531,0.001575,0.004921,0.002469,-0.005844,0.0,0.0,-0.004714,-0.005229,...,-0.013508,0.010073,-0.013631,0.007591,-0.009584,-0.014693,-0.002582,-0.004349,0.001630,-0.010800


In [6]:
stock_returns['^GSPC'].isna().sum()

0


# Experiment setup
- daily return을 기준으로 index tracking 설정
- 월별 리벨런싱 가정
- 2006/01/01 ~ 2022-12/30
- k act: 10% 20% 30% 40% 50% - 전체 주식의 % 

# Algorithm

## CARD

In [11]:
def card(X, y, kact, time_limit=1200, seed=42):
    #n number of row, day
    n, p = X.shape
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    
    objective = (1/n) * ((y - X @ beta) @ (y - X @ beta))
    #objective 설정
    m.setObjective(objective, GRB.MINIMIZE)
    #constraint 0: z의 설정 beta가 0이 아니면 1로 설정
    m.addConstr(beta <= z, name="c0")
    #constraint 1: full investment
    m.addConstr(beta.sum() == 1, name="c1")
    #constraine 2: cardinality constraint
    m.addConstr(z.sum() <= kact, name="c2")
    
    m.optimize()
    
    return beta.X


In [12]:
stock_stocks = stock_returns.drop(columns=['^GSPC'])
stock_index = stock_returns['^GSPC']

train_data = stock_stocks.iloc[:250]

# Filter out columns with NaN values in the current window
valid_assets_train = train_data.dropna(axis=1)
valid_assets = valid_assets_train.columns.intersection(valid_assets_train.columns)

X_train = valid_assets_train[valid_assets].values
y_train = stock_index.iloc[:250].values

In [13]:
# # CARD
beta_card = card(X_train, y_train, kact=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)


Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 535 rows, 1066 columns and 2132 nonzeros
Model fingerprint: 0x0f2c00e8
Model has 142310 quadratic objective terms
Variable types: 533 continuous, 533 integer (533 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-06, 2e-02]
  QObjective range [4e-09, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0007714
Presolve time: 0.05s
Presolved: 535 rows, 1066 columns, 2132 nonzeros
Presolved model has 142310 quadratic objective terms
Variable types: 533 con

In [10]:
non_negative_count

10

In [ ]:
beta_card

In [ ]:
# # CARD
beta_card_50 = card(X_train, y_train, kact=10, time_limit=60, seed=50)
beta_card_50

we checked if there are differnet seed for model, there is different beta(weight of portfolio)!

## ALASSO Algorithm
- make initial beta using OLS

In [64]:
def alasso(X, y, kact, initial_beta=None, nu=1, tol=1e-10000, time_limit = 1200):
    n, p = X.shape    
    # If no initial beta is provided, use ordinary least squares estimate
    if initial_beta is None:
            init_m = gp.Model()
            init_m.setParam('TimeLimit', time_limit)
            initial_beta = init_m.addMVar(shape=p, lb=0, name="initial_beta")
            init_objective = (1/n) * ((y - X @ initial_beta) @ (y - X @ initial_beta))
            #objective 설정
            init_m.setObjective(init_objective, GRB.MINIMIZE)
            #constraint 1: full investment
            init_m.addConstr(initial_beta.sum() == 1, name="c0")
            init_m.optimize()
            initial_beta = initial_beta.X

    # Calculate weights
    weights = 1 / (np.abs(initial_beta) ** nu)
    m = gp.Model()
#     m.setParam('TimeLimit', time_limit)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")

    objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + (weights @ beta)
    #objective 설정에 penalty term이 추가됨
    m.setObjective(objective, GRB.MINIMIZE)
    #constraint 0: z의 설정 beta가 0이 아니면 1로 설정
    m.addConstr(beta <= z, name="c0")
    #constraint 1: full investment
    m.addConstr(beta.sum() == 1, name="c1")
    #constraine 2: cardinality constraint
    m.addConstr(z.sum() <= kact, name="c2")
    m.optimize()
    return beta.X



In [65]:
# ALASSO
beta_alasso = alasso(X_train, y_train, kact=10)
non_negative_count = np.sum(beta_alasso > 0)
non_negative_count

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 533 columns and 533 nonzeros
Model fingerprint: 0xafe60f74
Model has 142310 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-06, 2e-02]
  QObjective range [4e-09, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.06s
Presolved: 1 rows, 533 columns, 533 nonzeros
Presolved model has 142310 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 250
 AA' NZ     : 3.138e+04
 Factor NZ  : 3.163e+04 (roughly 1 MB of memory)
 Factor Ops : 5.303e+06 (less 

1

In [66]:
non_negative_count

1

## Sorted LASSO- SLOPE
detail is from Sparse index clones via the sorted l1-Norm

In [71]:
def slope(X, y, kact, alpha=0.1, theta=0.1):
    """
    Solve the SLOPE optimization problem using Gurobi with an activation constraint.
    
    Parameters:
    X : array-like, shape (T, K)
        The return matrix of benchmark constituents.
    y : array-like, shape (T,)
        The benchmark returns.
    lambdas: regularization parameter vector for n features(followed instruction of experiment)
    kact : int
        Number of active variables (non-zero coefficients) allowed.
    alpha : float, optional
        Regularization parameter for the lambda sequence.
    theta : float, optional
        Decay rate for the lambda sequence.
        
    Returns:
    w_opt : array-like, shape (K,)
        Optimal weight vector.
    """

    # Dimensions
    T, K = X.shape

    # Lambda sequence
    # ppf is inverse function of normal distribution CDF 
    lambdas = np.array([alpha * norm.ppf(1 - (i * theta / (2 * K))) for i in range(1, K + 1)])

    # Initialize model
    model = gp.Model("SLOPE")
    
    # Variables
    w = model.addVars(K, lb=0, name="w")
    z = model.addVars(K, lb=0, name="z")
    u = model.addVars(K, lb=0, name="u")
    abs_w = model.addVars(K, lb=0, name="abs_w")

    # Constraints for absolute values and sorting
    model.addConstrs((z[i] >= w[i] for i in range(K)), "abs_pos")
    model.addConstrs((z[i] >= -w[i] for i in range(K)), "abs_neg")
    model.addConstrs((u[i] == gp.quicksum(z[j] for j in range(i+1)) for i in range(K)), "sort")
    
    # Activation constraint
    model.addConstr(gp.quicksum(abs_w[i] >= 1e-5 for i in range(K)) <= kact, "kact_limit")

    # Objective function
    obj = (1/(2*T)) * gp.quicksum((y[t] - gp.quicksum(X[t,j]*w[j] for j in range(K)))**2 for t in range(T)) + \
          gp.quicksum(lambdas[i]*u[i] for i in range(K))
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Solve model
    model.optimize()
    
    # Retrieve the optimal weights
    w_opt = np.array([w[i].x for i in range(K)])
    
    return w_opt

In [72]:
# SLOPE
beta_slope = slope(X_train, y_train, kact=10)
non_negative_count = np.sum(beta_slope > 0)
non_negative_count

TypeError: unsupported operand type(s) for +=: 'gurobipy.LinExpr' and 'TempConstr'

# SLOPE-SLC
1. Sorted LASSO의 변형 형태
2. compute for each group the median partial correlation of consituents and keep only groups which are including 75th percent quantile for the equity

In [ ]:
def slope_slc(X, y, lambdas, group_percentile=75, time_limit=1200):
    """
    SLOPE-SLC algorithm implementation using Gurobi.

    Parameters:
    X : numpy.ndarray
        Design matrix (n_samples, n_features)
    y : numpy.ndarray
        Response vector (n_samples,)
    lambdas : numpy.ndarray
        Regularization parameter vector (n_features,)
    group_percentile : float
        Percentile to select important groups (75 for equity, 25 for hedge funds)
    time_limit : int, optional
        Time limit for the optimization solver in seconds, default is 1200 seconds.

    Returns:
    numpy.ndarray
        Estimated coefficients (n_features,)
    """
    n, p = X.shape

    # Create a Gurobi model
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)

    # Define variables
    beta = m.addMVar(shape=p, lb=-GRB.INFINITY, name="beta")
    abs_beta = m.addMVar(shape=p, lb=0, name="abs_beta")

    # Objective function: 1/2 ||y - X @ beta||^2 + sum(lambda_i * abs(beta)_i)
    residuals = y - X @ beta
    obj = (1/2) * (residuals @ residuals) + lambdas @ abs_beta
    m.setObjective(obj, GRB.MINIMIZE)

    # Constraints
    # 1. abs_beta >= beta
    m.addConstr(abs_beta >= beta, name="abs_beta_pos")
    # 2. abs_beta >= -beta
    m.addConstr(abs_beta >= -beta, name="abs_beta_neg")

    # Optimize the model
    m.optimize()

    # Extract the solution
    beta_opt = beta.X

    # Calculate partial correlations
    residuals = y - X @ beta_opt
    partial_correlations = np.array([np.dot(X[:, i], residuals) for i in range(p)])

    # Group selection based on percentile
    threshold = np.percentile(np.abs(partial_correlations), group_percentile)
    important_groups = np.abs(partial_correlations) >= threshold

    # Keep only the most important groups and set others to zero
    beta_opt[~important_groups] = 0

    # Rescale the remaining coefficients so that they sum up to 1
    if np.sum(beta_opt) != 0:
        beta_opt = beta_opt / np.sum(beta_opt)

    return beta_opt


In [ ]:
# SLOPE-SLC
beta_slope_slc = slope_slc(X_train, y_train, kact=10)
non_negative_count = np.sum(beta_slope_slc > 0)
non_negative_count

## MSW_LASSO
detail is from High-dimensional Sprase index tracking based on a multi-step coonvex optimization approach

In [24]:
def msw_lasso(X, y, kact, max_iter=100, tol=1e-4, penalty='MCP', a=3.7):
    n, p = X.shape
    weights = np.ones(p)
    
    def p_lambda(beta_j, lam, a):
        if penalty == 'MCP':
            return lam * beta_j - (beta_j**2) / (2 * a) if beta_j <= a * lam else (a * lam**2) / 2
        elif penalty == 'SCAD':
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
    
    for _ in range(max_iter):
        m = gp.Model()
        beta = m.addMVar(shape=p, lb=0, name="beta")
        z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta
        m.setObjective(objective, GRB.MINIMIZE)
        m.addConstr(beta.sum() == 1, name="c0")
        m.addConstr(z.sum() <= kact, name="c1")
        m.addConstr(beta >= 0, name="c2")
        m.optimize()
        
        beta_value = beta.X
        new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
        
        if np.linalg.norm(new_weights - weights) < tol:
            break
        weights = new_weights
    
    return beta_value

In [73]:
# # msw_lasso
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10)
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

NameError: name 'msw_lasso' is not defined

# Backtest
## Experiment result 
- Turn over 비용
- Tracking error(for out of sample)
- Computing time(running time second)

In [18]:
def evaluate_portfolio(X, y, beta):
    tracking_error = np.sqrt(np.mean((y - X @ beta)**2))
    turnover = np.sum(np.abs(beta[1:] - beta[:-1])) / len(beta)
    return tracking_error, turnover

In [19]:
def rolling_window_backtest(stock_returns, stock_index, train_window, test_window, kact, lambdas):
    results = {'CARD': [], 'ALASSO': [], 'SLOPE-SLC': [], 'MSW-LASSO': []}
    n = len(stock_returns)
    
    for start in range(0, n - train_window - test_window + 1, test_window):
        train_data = stock_returns.iloc[start:start + train_window]
        test_data = stock_returns.iloc[start + train_window:start + train_window + test_window]
    
        # Filter out columns with NaN values in the current window
        valid_assets_train = train_data.dropna(axis=1)
        valid_assets_test = test_data.dropna(axis=1)
        valid_assets = valid_assets_train.columns.intersection(valid_assets_test.columns)
        
        X_train = valid_assets_train[valid_assets].values
        y_train = stock_index.iloc[start:start + train_window].values
        
        X_test = valid_assets_test[valid_assets].values
        y_test = stock_index.iloc[start + train_window:start + train_window + test_window].values
        # Skip if no valid assets are available
        if X_train.shape[1] == 0 or X_test.shape[1] == 0:
            continue
        
        # # CARD
        beta_card = card(X_train, y_train, kact)
        te_card, to_card = evaluate_portfolio(X_test, y_test, beta_card)
        results['CARD'].append({'date': stock_returns.index[start + train_window], 'te': te_card, 'to': to_card})
        
        # ALASSO
        beta_alasso = alasso(X_train, y_train, kact)
        te_alasso, to_alasso = evaluate_portfolio(X_test, y_test, beta_alasso)
        results['ALASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_alasso, 'to': to_alasso})
        
        # # SLOPE-SLC
        beta_slope_slc = slope_slc(X_train, y_train, kact, lambdas)
        te_slope_slc, to_slope_slc = evaluate_portfolio(X_test, y_test, beta_slope_slc)
        results['SLOPE-SLC'].append({'date': stock_returns.index[start + train_window], 'te': te_slope_slc, 'to': to_slope_slc})
        
        # # MSW-LASSO
        beta_msw_lasso = msw_lasso(X_train, y_train, kact)
        te_msw_lasso, to_msw_lasso = evaluate_portfolio(X_test, y_test, beta_msw_lasso)
        results['MSW-LASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso, 'to': to_msw_lasso})
        
        print(f"Window ending {stock_returns.index[start + train_window].date()}:")
        print(f"CARD: TE={te_card:.4f}, TO={to_card:.4f}")
        print(f"ALASSO: TE={te_alasso:.4f}, TO={to_alasso:.4f}")
        print(f"SLOPE-SLC: TE={te_slope_slc:.4f}, TO={to_slope_slc:.4f}")
        print(f"MSW-LASSO: TE={te_msw_lasso:.4f}, TO={to_msw_lasso:.4f}")
    
    return results


In [20]:
# Set parameters
kact = 10
lambdas = np.linspace(0.1, 1, stock_returns.shape[1]-1)  # Adjust as needed
train_window = 250
test_window =21

# DATA usage
X = stock_returns.drop(columns=['^GSPC'])
y = stock_returns['^GSPC']


In [21]:

# Perform the rolling window backtest
results = rolling_window_backtest(X, y, train_window, test_window, kact, lambdas)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 535 rows, 1066 columns and 2132 nonzeros
Model fingerprint: 0x0f2c00e8
Model has 142310 quadratic objective terms
Variable types: 533 continuous, 533 integer (533 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-06, 2e-02]
  QObjective range [4e-09, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0001483
Presolve time: 0.05s
Presolved: 535 rows, 1066 columns, 2132 nonzeros
Presolved model has 142310 quadratic objective terms
Variable types: 533 continuous, 533 integer (533 bin

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


H  855   820                       0.0000029    0.00000  96.1%   106   27s
H  893   840                       0.0000029    0.00000  96.1%   104   28s
   974   915    0.00000   32  122    0.00000    0.00000  96.1%   106   30s
H  995   888                       0.0000028    0.00000  95.9%   107   30s
H 1006   878                       0.0000027    0.00000  95.8%   108   32s
H 1008   873                       0.0000027    0.00000  95.7%   108   33s
  1057   975    0.00000   59  123    0.00000    0.00000  95.7%   110   35s


### 시각화

In [ ]:
# Convert results to DataFrame
df_results = {key: pd.DataFrame(val).set_index('date') for key, val in results.items()}

# Plot the results
plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['te'], label=f'{key} Tracking Error')

plt.title('Tracking Error Over Time')
plt.xlabel('Date')
plt.ylabel('Tracking Error')
plt.legend()
plt.show()

plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['to'], label=f'{key} Turnover')

plt.title('Turnover Over Time')
plt.xlabel('Date')
plt.ylabel('Turnover')
plt.legend()
plt.show()


# RMIT(Rank Meets Index tracking model)

1. RMIT model makes n samples of B(weight of porfolio) -> using n Regression model(with diffent randomn state or epoch)
2. for n num of B it test tracking error using train set(1month? or 12 month?)
3. for n num of B it test tracking error using valid set(1 month)
4. compare rank between 2,3 and add it's loss == rank Loss + origin loss 
5. repeat 1~3 which add rank loss, stop when beta does not change(small then epsilon) 
-> for last choose best beta(smallest beta) 

In [ ]:
import numpy as np
from scipy.stats import norm
import gurobipy as gp
from gurobipy import GRB
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

def rank_loss(rank_type, train_errors, valid_errors):
    if rank_type == "pointwise":
        return np.sum(np.abs(train_errors - valid_errors))
    elif rank_type == "pairwise":
        return np.sum((train_errors - valid_errors)**2)
    elif rank_type == "listwise":
        return np.sum((np.argsort(train_errors) - np.argsort(valid_errors))**2)
    else:
        raise ValueError("Invalid rank_type. Choose from 'pointwise', 'pairwise', 'listwise'.")

def optimize_portfolio(X_train, y_train, lambdas, kact, epsilon=1e-5):
    T, K = X_train.shape

    # Initialize model
    model = gp.Model("SLOPE")
    
    # Variables
    w = model.addVars(K, lb=-GRB.INFINITY, name="w")
    z = model.addVars(K, lb=0, name="z")
    u = model.addVars(K, lb=0, name="u")
    abs_w = model.addVars(K, vtype=GRB.BINARY, name="abs_w")

    # Constraints for absolute values and sorting
    model.addConstrs((z[i] >= w[i] for i in range(K)), "abs_pos")
    model.addConstrs((z[i] >= -w[i] for i in range(K)), "abs_neg")
    model.addConstrs((u[i] == gp.quicksum(z[j] for j in range(i+1)) for i in range(K)), "sort")
    
    # Activation constraint
    model.addConstr(gp.quicksum(abs_w[i] for i in range(K)) <= kact, "kact_limit")

    # Linking constraint for abs_w and w
    model.addConstrs((abs_w[i] >= w[i] / 1e-5 for i in range(K)), "link_pos")
    model.addConstrs((abs_w[i] >= -w[i] / 1e-5 for i in range(K)), "link_neg")
    model.addConstrs((abs_w[i] <= 1 for i in range(K)), "link_bin")

    # Objective function
    obj = (1/(2*T)) * gp.quicksum((y_train[t] - gp.quicksum(X_train[t,j]*w[j] for j in range(K)))**2 for t in range(T)) + \
          gp.quicksum(lambdas[i]*u[i] for i in range(K))
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Solve model
    model.optimize()
    
    # Retrieve the optimal weights
    w_opt = np.array([w[i].x for i in range(K)])
    
    return w_opt

def RMIT(X_train, y_train, X_valid, y_valid, kact, rank_type, alpha=0.1, theta=0.1, epsilon=1e-5, n_models=10, max_iter=100):
    T, K = X_train.shape
    lambdas = np.array([alpha * norm.ppf(1 - (i * theta / (2 * K))) for i in range(1, K + 1)])

    best_beta = None
    best_loss = float('inf')
    beta_change = float('inf')
    
    for iteration in range(max_iter):
        # 1. Generate n samples of B using n regression models
        B_samples = []
        for i in range(n_models):
            model = Lasso(alpha=alpha, random_state=i)
            model.fit(X_train, y_train)
            B_samples.append(model.coef_)

        # 2. Test tracking error using train set
        train_errors = []
        for B in B_samples:
            y_pred = X_train @ B
            train_errors.append(mean_squared_error(y_train, y_pred))

        # 3. Test tracking error using valid set
        valid_errors = []
        for B in B_samples:
            y_pred = X_valid @ B
            valid_errors.append(mean_squared_error(y_valid, y_pred))

        # 4. Compare rank between train and valid errors and add its loss
        rank_loss_value = rank_loss(rank_type, train_errors, valid_errors)

        # Optimize with added rank loss
        w_opt = optimize_portfolio(X_train, y_train, lambdas, kact, epsilon)
        y_train_pred = X_train @ w_opt
        origin_loss = mean_squared_error(y_train, y_train_pred)
        total_loss = origin_loss + rank_loss_value

        # 5. Repeat until beta does not change significantly
        if best_loss - total_loss > epsilon:
            best_loss = total_loss
            best_beta = w_opt
            beta_change = best_loss - total_loss
        else:
            break

    return best_beta

# Example usage:
# X_train = np.array([...])  # Training set feature matrix
# y_train = np.array([...])  # Training set target vector
# X_valid = np.array([...])  # Validation set feature matrix
# y_valid = np.array([...])  # Validation set target vector
# kact = 10  # Number of active variables
# rank_type = 'pointwise'  # Rank loss type ('pointwise', 'pairwise', 'listwise')
# alpha = 0.1  # Regularization parameter
# theta = 0.1  # Decay rate
# epsilon = 1e-5  # Convergence threshold
# n_models = 10  # Number of regression models
# max_iter = 100  # Maximum number of iterations
# best_beta = RMIT(X_train, y_train, X_valid, y_valid, kact, rank_type, alpha, theta, epsilon, n_models, max_iter)
